In [ ]:
# HW2: Information Gain
# Group Members: Steve Sommer
# Due: Oct 27, 2023 11:59 PM

In [35]:
import math

In [36]:
# Create a function to open input file
# Return list of instances
def get_inst_from_file(filename):
    all_instances = []
    fin = open(filename,"r")
    fin.readline()

    for line in fin:
        all_instances.append(line.strip().split(","))
    
    fin.close()
    
    return all_instances

In [37]:
# Print first 10 instances
titanic_instances = get_inst_from_file("titanic_new.csv")
print(titanic_instances[:10])

[['3', 'male', 'A', '1', '0', '0'], ['1', 'female', 'A', '1', '0', '1'], ['3', 'female', 'A', '0', '0', '1'], ['1', 'female', 'A', '1', '0', '1'], ['3', 'male', 'A', '0', '0', '0'], ['3', 'male', 'A', '0', '0', '0'], ['1', 'male', 'A', '0', '0', '0'], ['3', 'male', 'C', '3', '1', '0'], ['3', 'female', 'A', '0', '2', '1'], ['2', 'female', 'C', '1', '0', '1']]


In [38]:
# Create a function to open attributes file
# Return list of attribute values, along with attrib name

def get_attribs_from_file(filename):
    attribs = []
    fin = open(filename,"r")
    
    for line in fin:
        words = line.strip().split()
        attribs.append(words[1:])
        
    return attribs

In [39]:
# Print attrib info
titanic_attribs = get_attribs_from_file("attribs")
print(titanic_attribs)

[['1', '2', '3'], ['male', 'female'], ['A', 'C'], ['0', '1', '2', '3', '4', '5', '8'], ['0', '1', '2', '3', '4', '5', '6'], ['0', '1']]


In [99]:
# TODO: Write a function that returns class's entropy: H(Y)
# INPUT:   * List of instances, with label in last column
#          * value for first class (first possible value for label)
#          * value for second class (second possible value for label)
# RETURN:  Single float value for entropy
def get_entropy(instances, class0, class1):
    
    # HINT: Get counts for both classes
    count_class0 = sum(1 for instance in instances if instance[-1] == class0)
    count_class1 = sum(1 for instance in instances if instance[-1] == class1)
    
    # HINT: Get class count for both classes
    total_count = len(instances)
    
    # Calculate the probability of each class
    prob_class0 = count_class0 / total_count
    prob_class1 = count_class1 / total_count
    
    if prob_class0 <= 0 or prob_class1 <= 0:
        print(f"BUG: {prob_class0} {prob_class1} {total_count} {count_class0} {count_class1} ")

    # HINT: Be careful! Remember that log(0) is undefined (regardless of the log's base)
    entropy = -(prob_class0 * log2(prob_class0) + prob_class1 * log2(prob_class1))
    return entropy

# Test entropy function
all_entropy = get_entropy(titanic_instances,'0','1')
print(all_entropy)

0.9618806789594468


In [41]:
# Test entropy function
all_entropy = get_entropy(titanic_instances,'0','1')
print(all_entropy)

0.9618806789594468


In [95]:
# TODO: Write a function that returns conditional entropy: H(Y|X)
# INPUT:   * List of instances, with label in last column
#          * Feature index integer that indicates which feature to calculate I.G. for
#          * List of values feature can take on (comes from attribs file) 
#          * value for first class (first possible value for label)
#          * value for second class (second possible value for label)
# RETURN:  Return single float value for conditional entropy.
def get_cond_entropy(instances, feat_index, feat_vals, class0, class1):
    num_inst = len(instances)
    total_entropy = 0.0
    
    
    # HINT: Iterate over all feat_vals (i.e., all values the current feature--feat_index--can take on).
    for val in feat_vals:
    #for i in range(len(feat_vals)):
        
        # Get the current value from feat_vals using the index
        #val = feat_vals[i]
        
        # Filter instances where the feature (feat_index) has the current value (val)
        filtered_instances = [instance for instance in instances if instance[feat_index] == val]

        # HINT: Calculate probability of attribute taking on this value.
        prob_val = len(filtered_instances) / num_inst
        
        # HINT: You should be able to use your get_entropy() function,
        #        using the subset of instances where attribute takes on current
        #        value.
        conditional_entropy = get_entropy(filtered_instances, class0, class1)

        # Update the total conditional entropy
        total_entropy += prob_val * conditional_entropy

    return total_entropy
        

# Test conditional entropy
this_e = get_cond_entropy(titanic_instances, 2, ['A','C'], '0','1')
print("This e: ", this_e)

This e:  0.9551827486262512


In [47]:
# Test conditional entropy
this_e = get_cond_entropy(titanic_instances, 2, ['A','C'], '0','1')
print("This e: ", this_e)

This e:  0.9551827486262512


In [96]:
# TODO: Write a function that returns I(Y;X) = H(Y) - H(Y|X)
# INPUT:   * List of instances, with label in last column
#          * Feature index integer that indicates which feature to calculate I.G. for
#          * List of values feature can take on (comes from attribs file) 
#          * value for first class (first possible value for label)
#          * value for second class (second possible value for label)
# RETURN:  Single float value for info_gain on this feature

def get_info_gain(instances, feat_index, feat_vals, class0, class1):
    # HINT: you should be able to just call your entropy and conditional entropy
    #         functions you've already coded.
    
    entropyy = get_entropy(instances, class0, class1)
    cond_entropyy = get_cond_entropy(instances, feat_index, feat_vals, class0, class1)
    
    info_gain = entropyy - cond_entropyy
    return info_gain
    
# Test information gain
info_gain = get_info_gain(titanic_instances, 2, ['A', 'C'], '0', '1')
print("Information Gain: ", info_gain)

Information Gain:  0.006697930333195656


In [100]:
# Go thru all features and find corresponding IG measures.
# Be careful that you skip the last entry (class labels) in attrib array.
# Store them in a list to print later.

all_IGs = []
c1,c2 = titanic_attribs[len(titanic_attribs)-1]
print("c1,c2: ", c1,c2)  # Two class values

for i in range(len(titanic_attribs)-1):
    all_IGs.append(get_info_gain(titanic_instances, i, titanic_attribs[i], c1,c2))

c1,c2:  0 1
BUG: 1.0 0.0 5 5 0 


ValueError: math domain error

In [21]:
print(all_IGs)

[0.08286373372692346, 0.2168495048312652, 0.006697930333195656]
